In [27]:
import augmentor
import dataprovider3 as dp3
import h5py
import numpy as np
import os

## FIB-25 Validation Sample

In [28]:
home = os.path.expanduser('~')
fib25_path = os.path.join(home, 'Data_local/FIB-25/validation_sample')

In [29]:
fpath = os.path.join(fib25_path, 'img.h5')
with h5py.File(fpath, 'r') as f:
    img = f['/main'][...]

fpath = os.path.join(fib25_path, 'seg.h5')
with h5py.File(fpath, 'r') as f:
    seg = f['/main'][...]
    
fpath = os.path.join(fib25_path, 'msk.h5')
with h5py.File(fpath, 'r') as f:
    msk = f['/main'][...]

In [30]:
img = (img/255.).astype(np.float32)

In [31]:
# DataSet
dset = dp3.Dataset()
dset.add_data('img', img)
dset.add_data('seg', seg)
dset.add_mask('seg_mask', msk, loc=True)

In [32]:
# DataProvider
dims = (240,240,240)
spec = {'img': dims, 'seg': dims, 'seg_mask': dims}
dp = dp3.DataProvider(spec)
dp.add_dataset(dset)
dp.set_imgs(['img'])
dp.set_segs(['seg'])

In [33]:
# Augmentor
factor = (4,1,1)
aug = augmentor.SubsampleLabels(factor=factor)
print(aug)

SubsampleLabels(factor=[4 1 1])


In [34]:
dp.set_augment(aug)

In [35]:
sample = dp()

In [36]:
sample['img'].shape

(1, 240, 240, 240)

In [37]:
sample['seg'].shape

(1, 60, 240, 240)

In [38]:
sample['seg_mask'].shape

(1, 60, 240, 240)

In [39]:
import napari

simg = sample['img'][0::,2::4,0::1,0::1]*255
sseg = sample['seg']
smsk = sample['seg_mask']*255

viewer = napari.view_image(simg.astype(np.uint8), name='image')
viewer.add_labels(sseg, name='segmentation')
viewer.add_image(smsk, name='mask')
napari.run()